In [1]:
import requests

In [2]:
standings_url = "https://www.sports-reference.com/cbb/seasons/men/2024-school-stats.html" #all NCAA teams and their ranking and stats. Also has links to individual team pages

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)

In [6]:
standings_table = soup.select('table.stats_table')[0]

In [7]:
links = standings_table.find_all('a')

In [8]:
links = [l.get("href") for l in links] #grabs team page links

In [9]:
team_urls = [f"https://www.sports-reference.com{l}" for l in links] #team page links are releative url, convering to absolute

In [10]:
team_url = team_urls[0]

In [11]:
data = requests.get(team_url)

In [12]:
soup = BeautifulSoup(data.text)

In [13]:
game_logs = soup.select('div.inactive')[0]

In [14]:
links = game_logs.find_all('a')

In [15]:
links = [l.get("href") for l in links]

In [16]:
links

['/cbb/schools/abilene-christian/men/',
 '/cbb/schools/abilene-christian/men/2024.html',
 '/cbb/schools/abilene-christian/men/2024-schedule.html',
 '/cbb/schools/abilene-christian/men/2024-gamelogs.html',
 '/cbb/schools/abilene-christian/men/2024-starters.html',
 '/cbb/schools/abilene-christian/men/2024.html',
 '/cbb/schools/abilene-christian/men/2024-schedule.html',
 '/cbb/schools/abilene-christian/men/2024-gamelogs.html',
 '/cbb/schools/abilene-christian/men/2024-starters.html',
 '#all_roster',
 '#all_per_game_team',
 '#all_players_per_game',
 '#all_players_totals',
 '#all_players_per_min',
 '#all_players_per_poss',
 '#all_players_advanced',
 '#site_menu_link']

In [17]:
import pandas as pd

In [18]:
from io import StringIO

In [19]:
import html5lib

In [20]:
data = requests.get(f"https://www.sports-reference.com{links[3]}")

In [21]:
game_log = pd.read_html(StringIO(data.text), match = "Basic Game Log", flavor="bs4")[0]

In [22]:
game_log.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                   G               Date Unnamed: 2_level_1                Opp   
0                  1         2023-11-06                  @     Oklahoma State   
1                  2         2023-11-10                  @           NC State   
2                  3         2023-11-14                NaN       Prairie View   
3                  4         2023-11-17                  N     San Jose State   
4                  5         2023-11-19                  N            Fordham   

  Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0 School            \
                 W/L                 Tm                Opp     FG FGA   FG%   
0                  W                 64                 59     26  57  .456   
1                  L                 64                 84     20  57  .351   
2                  L                 74                 79     23  60  .383   
3                  W                 77                 71     25  59  .424   
4                  W                 59                 45     22  55  .400   

   ... Opponent                                        
   ...       FT FTA   FT% ORB TRB AST STL BLK TOV  PF  
0  ...        8  20  .400   6  32  13   9   2  12  16  
1  ...       20  24  .833  12  34  12   4   3   7  20  
2  ...       33  34  .971   8  34   7   8   5  14  23  
3  ...        7  12  .583   9  35  13   4   3   6  18  
4  ...       12  21  .571  16  41   3   5   3  19  16  

[5 rows x 40 columns]

In [23]:
game_log.columns = game_log.columns.droplevel()

In [24]:
game_log.head()

,G,Date,Unnamed: 2_level_1,Opp,W/L,Tm,Opp,FG,FGA,FG%,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,2023-11-06,@,Oklahoma State,W,64,59,26,57,.456,...,8,20,.400,6,32,13,9,2,12,16
1,2,2023-11-10,@,NC State,L,64,84,20,57,.351,...,20,24,.833,12,34,12,4,3,7,20
2,3,2023-11-14,NaN,Prairie View,L,74,79,23,60,.383,...,33,34,.971,8,34,7,8,5,14,23
3,4,2023-11-17,N,San Jose State,W,77,71,25,59,.424,...,7,12,.583,9,35,13,4,3,6,18
4,5,2023-11-19,N,Fordham,W,59,45,22,55,.400,...,12,21,.571,16,41,3,5,3,19,16


In [25]:
# Now scaling to all teams

In [26]:
years = list(range(2024, 2022, -1))

In [27]:
years

[2024, 2023]

In [28]:
all_matches = []

In [29]:
import time

In [30]:
standings_url = "https://www.sports-reference.com/cbb/seasons/men/2024-school-stats.html"

In [31]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    team_urls = [f"https://www.sports-reference.com{l}" for l in links]

    standings_url = "https://www.sports-reference.com/cbb/seasons/men/2023-school-stats.html"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-3].replace("-", " ")

        data = requests.get(team_url)
        soup = BeautifulSoup(data.text)
        game_logs = soup.select('div.inactive')[0]
        links = game_logs.find_all('a')
        links = [l.get("href") for l in links]
        data = requests.get(f"https://www.sports-reference.com{links[3]}")
        game_log = pd.read_html(StringIO(data.text), match = "Basic Game Log", flavor="bs4")[0]
        game_log.columns = game_log.columns.droplevel()

        game_log["Season"] = year
        game_log["Team"] = team_name
        all_matches.append(game_log)

        time.sleep(5)

In [32]:
match_df = pd.concat(all_matches)

In [33]:
match_df.columns = [c.lower() for c in match_df.columns]

In [34]:
match_df.to_csv("NCAA_Data.csv")